# Loop through watershed FCs and remove holes inside polygons.  Updates the input feature's geometry <b>DOES NOT PRODUCE  OUTPUT</b> so make copy of data if necessary.
Change data_dir = "path to your AKSSF folder containing regional subfolders"
* **NOTE - this will capture all polygon features with 'wtd' in the name**

In [7]:
#Def function to remove holes
def RemovePolygonHoles_management(in_fc, threshold=0.0):
    """
    The function removes holes from a polygon feature class.
    If threshold is given, only the holes smaller than the threshold will be removed.
    If no threshold is given, it removes all holes.
    in_fc is a polygon feature class.
    threshold is numeric.
    """
    import time
    import datetime
    # Start timing function
    rhprocessStart = time.time()
    rhprocessStartdt = datetime.datetime.now()
    desc = arcpy.Describe(in_fc)
    if desc.featureType !="Simple":
        print ("Invalid data type. Simple type required.")
        return
    else:
        if desc.shapeType != "Polygon":
            print ("The input is supposed to be a Polygon FeatureClass or Shapefile.")
            return
    if threshold < 0.0:
        threshold = 0.0
    print(f'Begin filling holes in {in_fc}')
    with arcpy.da.UpdateCursor(in_fc, ["SHAPE@"]) as updateCursor:
        for updateRow in updateCursor:
            shape = updateRow[0]
            new_shape = arcpy.Array()
            for part in shape:
                new_part = arcpy.Array()
                if threshold > 0:
                    #find None point in shape part
                    #in arcpy module, a None point is used to seperate exterior and interior vertices
                    null_point_index = []
                    for i in range(len(part)):
                        if part[i] == None:
                            null_point_index.append(i)
                    #if interior vertices exist, create polygons and compare polygon shape area to given threshold
                    #if larger, keep vertices, else, dismiss them
                    if len(null_point_index) > 0:
                        for k in range(0, null_point_index[0]):
                            new_part.add(part[k])
                        for i in range(len(null_point_index)):
                            pointArray = arcpy.Array()
                            #determine if the None point is the last one
                            if i+1 < len(null_point_index):
                                for j in range(null_point_index[i] + 1, null_point_index[i+1]):
                                    pointArray.add(part[j])
                            else:
                                for j in range(null_point_index[i] + 1, len(part)):
                                    pointArray.add(part[j])
                            #create a polygon to check shape area against the given threshold
                            inner_poly = arcpy.Polygon(pointArray)
                            #if larger than threshold, then add to the new part Array
                            if inner_poly.area > threshold:
                                if i+1 < len(null_point_index):
                                    for k in range(null_point_index[i], null_point_index[i+1]):
                                        new_part.add(part[k])
                                else:
                                    for k in range(null_point_index[i], len(part)):
                                        new_part.add(part[k])
                        new_shape.add(new_part)
                    #if interior does not exist, add the whole part
                    else:
                        new_shape.add(part)
                else:
                    #get the first None point index
                    first_null_point_index = 0
                    for i in range(len(part)):
                        if part[i] == None:
                            first_null_point_index = i
                            break
                    if first_null_point_index == 0:
                        new_shape.add(part)
                    else:
                        for j in range(first_null_point_index):
                            new_part.add(part[j])
                        new_shape.add(new_part)
            if len(new_shape) > 0:
                new_poly = arcpy.Polygon(new_shape)
                updateRow[0] = new_poly
                updateCursor.updateRow(updateRow)
    # End timing
    rhprocessEnd = time.time()
    rhprocessElapsed = int(rhprocessEnd - rhprocessStart)
    rhprocessSuccess_time = datetime.datetime.now()

    # Report success
    print(f'Remove holes for {in_fc} completed at {rhprocessSuccess_time.strftime("%Y-%m-%d %H:%M")} '
          f'(Elapsed time: {datetime.timedelta(seconds=rhprocessElapsed)})')
    print('----------')

In [ ]:
import arcpy
import os
import datetime
import time
# data_dir = r"D:\GIS_Temp\AKSSF"
# Set data_dir equal to folder containing AKSSF regional subfolders containing GDBs and raster datasets
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Seperate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
rois = nhdplus_dat + tauDem_dat

for roi in rois:
    # Loop through regional folders
    for region in regions:
        if roi in str(region):
            print(f'Updating Watersheds for {roi} in {region} folder')
            # Set workspace to region folder
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            walk = arcpy.da.Walk(region, datatype = 'FeatureClass')
            for dirpath, dirnames, filenames in walk:
                # loop through watershed datasets
                for filename in filenames:
                    if 'wtd' in filename:
                        rmtimestart = time.time()
                        print(f" {filename} watershed identified")
                        wtd = os.path.join(dirpath, filename)
                        print(wtd)
                        RemovePolygonHoles_management(wtd, threshold = 0.0)
# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')